In [1]:
import pandas as pd
import numpy as np
import pymongo as pymongo
import json
from pandas.io.json import json_normalize

In [2]:
def safe_div(x,y):
    if y == 0:
        return 0
    return x / y

In [3]:
client = pymongo.MongoClient("idenmon.zapto.org", 888)
db = client.prodLaboratorio
pruebas = db.joinedLabApp


In [4]:
dfGlobalOrig = pd.DataFrame(list(pruebas.find()))
lGlobal=list(pruebas.find())

In [5]:
dfGlobal=pd.DataFrame(lGlobal)
dfGlobal=dfGlobalOrig
#dfGlobal.head()

In [6]:
g=globals()
proteinas =['CFP10',  'ESAT6',  'RV1681',  'P24', 'TB']
for proteina in proteinas:
    g['{0}Counts'.format(proteina)] = dfGlobal[proteina+'_Comp'].groupby(dfGlobal['appVersion']).value_counts()

In [7]:
versiones = dfGlobal.appVersion.unique() #automatizarlo para extraer las versiones del DataFrame
mezclaVersionesProteinas = [(ver,prot) for ver in versiones for prot in proteinas]
d={}
#for ver, prot in mezclaVersionesProteinas:
for ver in versiones:
    v = ver.replace('.','')
    for prot in proteinas:
        g['{0}'.format(prot)]={}
        resProt = g['{0}Counts'.format(prot)].loc[ver]
        g['sens_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0) + resProt.get('FN',0)))
        #g['prec_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0)+resProt.get('FP',0)))
        g['specif_{0}'.format(prot)]= safe_div(resProt.get('TN',0), (resProt.get('TN',0)+resProt.get('FP',0)))
        g['posPV_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0)+resProt.get('FP',0)))
        g['negPV_{0}'.format(prot)]= safe_div(resProt.get('TN',0),(resProt.get('TN',0)+resProt.get('FN',0)))
        g['{0}'.format(prot)].update({'Sensitivity': g['sens_{0}'.format(prot)] })
        #g['{0}'.format(prot)].update({'Prescition': g['prec_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Specificity': g['specif_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Positive Predictive Value': g['posPV_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Negative Predictive Value': g['negPV_{0}'.format(prot)] })
        totalPruebas = resProt.get('TP',0)+resProt.get('FP',0)+resProt.get('TN',0)+resProt.get('FN',0)+resProt.get('IND',0)
        g['{0}'.format(prot)].update({'#TotalPruebas': totalPruebas})
        g['{0}'.format(prot)].update({'#TP': resProt.get('TP',0) })
        g['{0}'.format(prot)].update({'#FP': resProt.get('FP',0) })
        g['{0}'.format(prot)].update({'#TN': resProt.get('TN',0) })
        g['{0}'.format(prot)].update({'#FN': resProt.get('FN',0) })
        g['{0}'.format(prot)].update({'#Indet': resProt.get('IND',0)})
        g['{0}'.format(prot)].update({'#Invalidos': resProt.get('INVALID',0)})
        d.update({prot: g['{0}'.format(prot)]})
    g['df{0}'.format(v)] = pd.DataFrame.from_dict(d, orient='Index')

In [8]:
for i,ver in enumerate(versiones):
    nver = ver.replace('.','')
    g['df{0}T'.format(nver)] =g['df{0}'.format(nver)].T
    print('Resultados para version ',ver)
    print(g['df{0}T'.format(nver)])
    print('-'*75)


Resultados para version  0.1.0
                                CFP10       ESAT6         P24      RV1681  \
Sensitivity                  0.519481    0.645161    0.781659    0.830769   
Specificity                  0.935829    0.915423    1.000000    0.944724   
Positive Predictive Value    0.769231    0.701754    1.000000    0.830769   
Negative Predictive Value    0.825472    0.893204    0.137931    0.944724   
#TotalPruebas              315.000000  315.000000  315.000000  315.000000   
#TP                         40.000000   40.000000  179.000000   54.000000   
#FP                         12.000000   17.000000    0.000000   11.000000   
#TN                        175.000000  184.000000    8.000000  188.000000   
#FN                         37.000000   22.000000   50.000000   11.000000   
#Indet                      51.000000   52.000000   78.000000   51.000000   
#Invalidos                  11.000000   11.000000   11.000000   11.000000   

                                   TB  
Sens

In [9]:
df020T = df020.T
df010T = df010.T
df021T = df021.T
df031T = df031.T
dfComp = df010T.join(df020T,lsuffix='__v0.1.0', rsuffix='__v0.2.0')
dfComp

,CFP10__v0.1.0,ESAT6__v0.1.0,P24__v0.1.0,RV1681__v0.1.0,TB__v0.1.0,CFP10__v0.2.0,ESAT6__v0.2.0,P24__v0.2.0,RV1681__v0.2.0,TB__v0.2.0
Sensitivity,0.519481,0.645161,0.781659,0.830769,0.986486,0.0,0.75,1.0,1.000000,1.000000
Specificity,0.935829,0.915423,1.000000,0.944724,0.837349,0.0,0.50,0.5,0.000000,0.000000
Positive Predictive Value,0.769231,0.701754,1.000000,0.830769,0.730000,0.0,0.75,0.8,0.666667,0.666667
Negative Predictive Value,0.825472,0.893204,0.137931,0.944724,0.992857,0.0,0.50,1.0,0.000000,0.000000
#TotalPruebas,315.000000,315.000000,315.000000,315.000000,315.000000,6.0,6.00,6.0,6.000000,6.000000
#TP,40.000000,40.000000,179.000000,54.000000,73.000000,0.0,3.00,4.0,4.000000,4.000000
#FP,12.000000,17.000000,0.000000,11.000000,27.000000,2.0,1.00,1.0,2.000000,2.000000
#TN,175.000000,184.000000,8.000000,188.000000,139.000000,0.0,1.00,1.0,0.000000,0.000000
#FN,37.000000,22.000000,50.000000,11.000000,1.000000,4.0,1.00,0.0,0.000000,0.000000
#Indet,51.000000,52.000000,78.000000,51.000000,75.000000,0.0,0.00,0.0,0.000000,0.000000


### Para ver estadisticas por version de la app tomando solamente la primera version de la fotografia tomada

In [54]:
df31 = dfGlobal.loc[dfGlobal['appVersion']=='0.3.1']
df31Red = df31.drop_duplicates(subset='qr', keep="first")
dfGlobalSinRepetidos = df31Red

verRed = []
for ver in versiones:
    verRed.append(ver)
verRed.remove('0.2.0')
verRed.remove('0.1.0')
verRed.remove('0.2.1')

for proteina in proteinas:
    g['{0}Counts0'.format(proteina)] = dfGlobalSinRepetidos[proteina+'_Comp'].groupby(dfGlobalSinRepetidos['appVersion']).value_counts()
    
d={}
#for ver, prot in mezclaVersionesProteinas:
for ver in verRed:
    v = ver.replace('.','')
    for prot in proteinas:
        g['{0}'.format(prot)]={}
        resProt = g['{0}Counts0'.format(prot)].loc[ver]
        g['sens_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0) + resProt.get('FN',0)))
        #g['prec_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0)+resProt.get('FP',0)))
        g['specif_{0}'.format(prot)]= safe_div(resProt.get('TN',0), (resProt.get('TN',0)+resProt.get('FP',0)))
        g['posPV_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0)+resProt.get('FP',0)))
        g['negPV_{0}'.format(prot)]= safe_div(resProt.get('TN',0),(resProt.get('TN',0)+resProt.get('FN',0)))
        g['{0}'.format(prot)].update({'Sensitivity': g['sens_{0}'.format(prot)] })
        #g['{0}'.format(prot)].update({'Prescition': g['prec_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Specificity': g['specif_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Positive Predictive Value': g['posPV_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Negative Predictive Value': g['negPV_{0}'.format(prot)] })
        totalPruebas = resProt.get('TP',0)+resProt.get('FP',0)+resProt.get('TN',0)+resProt.get('FN',0)+resProt.get('IND',0)
        g['{0}'.format(prot)].update({'#TotalPruebas': totalPruebas})
        g['{0}'.format(prot)].update({'#TP': resProt.get('TP',0) })
        g['{0}'.format(prot)].update({'#FP': resProt.get('FP',0) })
        g['{0}'.format(prot)].update({'#TN': resProt.get('TN',0) })
        g['{0}'.format(prot)].update({'#FN': resProt.get('FN',0) })
        g['{0}'.format(prot)].update({'#Indet': resProt.get('IND',0)})
        g['{0}'.format(prot)].update({'#Invalidos': resProt.get('INVALID',0)})
        d.update({prot: g['{0}'.format(prot)]})
    g['df{0}'.format(v)] = pd.DataFrame.from_dict(d, orient='Index')
    
    
    
for i,ver in enumerate(verRed):
    nver = ver.replace('.','')
    g['df{0}T'.format(nver)] =g['df{0}'.format(nver)].T
    print('Resultados para versión ',ver,' sin imagenes repetidas')
    print(g['df{0}T'.format(nver)])
    print('-'*75)   



Resultados para versión  0.3.1  sin imagenes repetidas
                                CFP10       ESAT6         P24      RV1681  \
Sensitivity                  0.466667    0.674419    0.907895    0.755556   
Specificity                  0.987013    0.962025    1.000000    0.961039   
Positive Predictive Value    0.954545    0.906250    1.000000    0.918919   
Negative Predictive Value    0.760000    0.844444    0.867925    0.870588   
#TotalPruebas              122.000000  122.000000  122.000000  122.000000   
#TP                         21.000000   29.000000   69.000000   34.000000   
#FP                          1.000000    3.000000    0.000000    3.000000   
#TN                         76.000000   76.000000   46.000000   74.000000   
#FN                         24.000000   14.000000    7.000000   11.000000   
#Indet                       0.000000    0.000000    0.000000    0.000000   
#Invalidos                   4.000000    4.000000    4.000000    4.000000   

                    

### Calculo de estadísticas por enfermedad